In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# Read the CSV file
training_data = pd.read_csv("data/training_dataPhil.csv", index_col=0)

training_data.head()

,sentence,difficulty,lexical_complexite,note_orthographe,char_length,word_length,type_token_ratio,sentence_length,avg_word_length,complexite_texte,...,DET,PRON,NUM,NOUN,INTJ,ADP,ADJ,VERB,PROPN,SCONJ
id,,,,,,,,,,,,,,,,,,,,,
0,Les coûts kilométriques réels peuvent diverger...,4,0.194007,1.000000,0.160077,0.140152,0.467105,0.140152,0.339713,0.244565,...,0.066667,0.000000,0.000000,0.311111,0.0,0.288889,0.066667,0.088889,0.0000,0.0
1,"Le bleu, c'est ma couleur préférée mais je n'a...",0,0.082334,1.000000,0.036990,0.041667,1.000000,0.041667,0.204545,0.086957,...,0.187500,0.125000,0.000000,0.125000,0.0,0.000000,0.000000,0.125000,0.0625,0.0
2,Le test de niveau en français est sur le site ...,0,0.088078,0.769231,0.039541,0.045455,0.826923,0.045455,0.195804,0.081522,...,0.200000,0.000000,0.000000,0.400000,0.0,0.266667,0.000000,0.066667,0.0000,0.0
3,Est-ce que ton mari est aussi de Boston?,0,0.062664,1.000000,0.022959,0.026515,1.000000,0.026515,0.193182,0.054348,...,0.000000,0.100000,0.000000,0.300000,0.0,0.100000,0.000000,0.100000,0.1000,0.1
4,"Dans les écoles de commerce, dans les couloirs...",2,0.184993,1.000000,0.130740,0.125000,0.602941,0.125000,0.288770,0.228261,...,0.095238,0.047619,0.047619,0.238095,0.0,0.261905,0.047619,0.095238,0.0000,0.0


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AutoModelForSequenceClassification, AutoTokenizer
from torch.optim import Adam
import torch.nn.functional as F

In [4]:
from sentence_transformers import SentenceTransformer

# Charger le modèle CamemBERT
model_name = "dangvantuan/sentence-camembert-large"
#model = SentenceTransformer(model_name)

In [5]:
# Préparation des données
sentences = training_data['sentence'].tolist()
labels = training_data['difficulty'].tolist()

In [6]:
# Initialisation du tokeniseur et du modèle
tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = CamembertForSequenceClassification.from_pretrained(model_name, num_labels=6)
tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at dangvantuan/sentence-camembert-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


NE PAS RUN !!!!!!

In [7]:
# Récupération des tensors nécessaires du BatchEncoding
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Create a training dataset
train_dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, torch.tensor(labels))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Entraînement du modèle
for epoch in range(6):
    model.train()
    total_loss = 0

    # Utilisation de tqdm pour la barre de progression
    for batch in tqdm(train_loader, desc=f"Époque {epoch+1}/{6}"):
        input_ids_batch, attention_mask_batch, labels_batch = batch

        # Construire le dictionnaire d'entrée pour le modèle
        inputs = {'input_ids': input_ids_batch, 'attention_mask': attention_mask_batch, 'labels': labels_batch}
           
        outputs = model(**inputs)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Affichage de la perte moyenne pour l'époque
    print(f"Loss moyenne pour l'époque: {total_loss / len(train_loader)}")

Époque 1/6: 100%|██████████| 300/300 [3:13:39<00:00, 38.73s/it]  


Loss moyenne pour l'époque: 1.2588747866948446


Époque 2/6: 100%|██████████| 300/300 [3:11:17<00:00, 38.26s/it]  


Loss moyenne pour l'époque: 0.8929875550667444


Époque 3/6: 100%|██████████| 300/300 [3:08:05<00:00, 37.62s/it]  


...

Loss moyenne pour l'époque: 0.4309373359878858


Époque 5/6: 100%|██████████| 300/300 [3:09:42<00:00, 37.94s/it]  


Loss moyenne pour l'époque: 0.31548847556114196


Époque 6/6: 100%|██████████| 300/300 [3:09:30<00:00, 37.90s/it]  

Loss moyenne pour l'époque: 0.18432556837797165


torch.save(model.state_dict(), "save4/modele.pth")

# Après l'entraînement de chaque époque
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,  # Vous pouvez sauvegarder la perte moyenne ou la dernière perte enregistrée
    # Ajoutez d'autres métriques si nécessaire
}, "save4/complet.pth")

UTILISER LE MODEL POST-TRAIN











In [7]:
from transformers import CamembertForSequenceClassification, CamembertTokenizer
import torch

# Charger le tokenizer et le modèle pré-entraîné
model_name = "dangvantuan/sentence-camembert-large"

tokenizer = CamembertTokenizer.from_pretrained("dangvantuan/sentence-camembert-large")
model = CamembertForSequenceClassification.from_pretrained("dangvantuan/sentence-camembert-large", num_labels=6)

# Charger l'état sauvegardé de votre modèle post-entraîné
model.load_state_dict(torch.load("save4/complet.pth")['model_state_dict'])

# Mettre le modèle en mode évaluation
model.eval()

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at dangvantuan/sentence-camembert-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-23): 24 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=1024, out_features=10

In [8]:
import numpy as np
sentences = training_data['sentence'].tolist()
labels = training_data['difficulty'].tolist()

# Initialiser une liste pour stocker les caractéristiques extraites
all_features = []

# Boucle sur les données avec une barre de progression
for sentence in tqdm(sentences, desc="Processing sentences"):
    # Tokenisation et extraction de caractéristiques pour chaque phrase
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    features = outputs.logits.squeeze().detach().numpy()
    
    # Ajouter les caractéristiques à la liste
    all_features.append(features)

# Concaténer les caractéristiques en un seul tableau numpy
#all_features = np.concatenate(all_features, axis=0)

# Vérifier la forme des caractéristiques extraites
#print("Shape of features:", all_features.shape)

Processing sentences: 100%|██████████| 4800/4800 [12:46<00:00,  6.26it/s]


In [9]:
len(all_features)

4800

In [10]:

training_data_x = training_data.drop(columns=["difficulty", "sentence",'note_orthographe'])
combined_features = np.concatenate((all_features, training_data_x), axis=1)


In [12]:
from sklearn.model_selection import train_test_split

X = combined_features  # Les embeddings camembert
y = training_data["difficulty"]  # Les niveaux de difficulté (étiquettes)

# Effectuez la division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.svm import SVC

# Créez un modèle SVM (classification)
svm_model = SVC(C=10, gamma='scale', kernel='rbf')

# Entraînez le modèle sur l'ensemble d'entraînement
svm_model.fit(X_train, y_train)

# Évaluez la performance du modèle sur l'ensemble de test
accuracy = svm_model.score(X_test, y_test)
print("Précision du modèle SVM :", accuracy)


Précision du modèle SVM : 0.9697916666666667


In [28]:
test_data = pd.read_csv("data/unlabelled_test_dataPhil.csv", index_col=0)
test_data2 = pd.read_csv("data/unlabelled_test_data.csv", index_col=0)

test_data = test_data.drop(columns=['note_orthographe'])

In [29]:
test_data2.head()

,sentence
id,
0,Nous dûmes nous excuser des propos que nous eû...
1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,"Et, paradoxalement, boire froid n'est pas la b..."
3,"Ce n'est pas étonnant, car c'est une saison my..."
4,"Le corps de Golo lui-même, d'une essence aussi..."


In [31]:
import numpy as np
sentences = test_data['sentence'].tolist()

# Initialiser une liste pour stocker les caractéristiques extraites
all_features2 = []

# Boucle sur les données avec une barre de progression
for sentence in tqdm(sentences, desc="Processing sentences"):
    # Tokenisation et extraction de caractéristiques pour chaque phrase
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    features = outputs.logits.squeeze().detach().numpy()
    
    # Ajouter les caractéristiques à la liste
    all_features2.append(features)

# Concaténer les caractéristiques en un seul tableau numpy
#all_features = np.concatenate(all_features, axis=0)

# Vérifier la forme des caractéristiques extraites
#print("Shape of features:", all_features.shape)

Processing sentences: 100%|██████████| 1200/1200 [03:10<00:00,  6.31it/s]


In [32]:

test_data_x = test_data.drop(columns=["sentence",])
combined_features = np.concatenate((all_features2, test_data_x), axis=1)

In [33]:

# Prédictions
predicted_labels = svm_model.predict(combined_features)  # Assurez-vous d'extraire les caractéristiques des nouvelles phrases de la même manière que précédemment

# Ajouter les prédictions au DataFrame
test_data2['difficulty'] = predicted_labels

# Afficher le DataFrame mis à jour
print(test_data2.head())

                                             sentence  difficulty
id                                                               
0   Nous dûmes nous excuser des propos que nous eû...           5
1   Vous ne pouvez pas savoir le plaisir que j'ai ...           2
2   Et, paradoxalement, boire froid n'est pas la b...           3
3   Ce n'est pas étonnant, car c'est une saison my...           1
4   Le corps de Golo lui-même, d'une essence aussi...           5


In [34]:
test_data2 = test_data2.drop(columns=['sentence'])
difficulty_mapping = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

# Remplacez les valeurs dans la colonne "difficulty"
test_data2['difficulty'] = test_data2['difficulty'].map(difficulty_mapping)

In [36]:
print(test_data2.head(30))


   difficulty
id           
0          C2
1          B1
2          B2
3          A2
4          C2
5          C2
6          A2
7          A2
8          C1
9          A2
10         A2
11         A2
12         B2
13         C1
14         A1
15         A2
16         B2
17         A2
18         A2
19         A1
20         C2
21         C1
22         C1
23         C1
24         B1
25         C2
26         A1
27         A1
28         C2
29         B1


In [37]:
test_data2.shape

(1200, 1)

In [38]:
test_data2.to_csv('philippe+features.csv', index=True)


In [ ]:
# install Kaggle
! pip install kaggle

In [ ]:
# Charger le modèle post-entraînement
model = CamembertForSequenceClassification.from_pretrained("dangvantuan/sentence-camembert-large", num_labels=6)

# Tokeniser la phrase
input_ids = tokenizer(sentences, return_tensors="pt")["input_ids"]
attention_mask = tokenizer(sentences, return_tensors="pt")["attention_mask"]

# Passez la phrase au modèle
model.eval()  # Définir le modèle en mode évaluation
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Extraire les représentations tokenisées des sorties du modèle
tokenized_text = outputs.last_hidden_state

# Charger un autre modèle
new_model = ...

# Entraînez le nouveau modèle sur les représentations tokenisées
new_model.fit(tokenized_text, labels)

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Charger le modèle post-entraînement
model_name = "dangvantuan/sentence-camembert-large"
model = AutoModel.from_pretrained(model_name)

# Supposons que training_data contient vos données avec les phrases
phrases = training_data['sentence'].tolist()

# Générer des embeddings pour chaque phrase
bert_embeddings = []
for phrase in phrases:
    # Tokeniser la phrase
    input_ids = tokenizer(phrase, return_tensors="pt")["input_ids"]
    attention_mask = tokenizer(phrase, return_tensors="pt")["attention_mask"]

    # Passez la phrase au modèle
    model.eval()  # Définir le modèle en mode évaluation
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Obtenir les prédictions du modèle
    predictions = model.predict(input_ids=input_ids, attention_mask=attention_mask)

    # Calculer les embeddings
    bert_embeddings.append(predictions)





# Charger le modèle
model_name = "dangvantuan/sentence-camembert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Modèle pré-entraîné Sentence-CamemBERT-Large en français

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

# Charger le modèle Sentence CamemBERT
model_name = "dangvantuan/sentence-camembert-large"
model = SentenceTransformer(model_name)


# Supposons que training_data contient vos données avec les phrases
phrases = training_data['sentence'].tolist()

# Générer des embeddings pour chaque phrase
bert_embeddings = model.encode(phrases)

# Les embeddings sont maintenant stockés dans la variable 'embeddings'

No sentence-transformers model found with name /Users/phil/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [4]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model_name = "dangvantuan/sentence-camembert-large"
sentence_model = SentenceTransformer(model_name)

# Define a simple classifier
class AdvancedClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(AdvancedClassifier, self).__init__()
        # Définir les couches du réseau
        self.fc1 = nn.Linear(embedding_dim, 512)  # Première couche cachée
        self.relu = nn.ReLU()  # Fonction d'activation ReLU
        self.fc2 = nn.Linear(512, 128)  # Deuxième couche cachée
        self.fc3 = nn.Linear(128, num_classes)  # Couche de sortie

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Activation de la première couche cachée
        x = self.relu(self.fc2(x))  # Activation de la deuxième couche cachée
        x = self.fc3(x)  # Couche de sortie
        return x

# Utiliser le classificateur avancé
advanced_classifier = AdvancedClassifier(embedding_dim=1024, num_classes=6)

# Mettre à jour CombinedModel pour utiliser le nouveau classificateur
class CombinedModel(nn.Module):
    def __init__(self, sentence_model, classifier):
        super(CombinedModel, self).__init__()
        self.sentence_model = sentence_model
        self.classifier = classifier

    def forward(self, sentences):
        embeddings = self.sentence_model.encode(sentences, convert_to_tensor=True)
        return self.classifier(embeddings)

combined_model = CombinedModel(sentence_model, advanced_classifier)



No sentence-transformers model found with name /Users/phil/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [5]:
from sklearn.model_selection import train_test_split
y = training_data["difficulty"]
# Supposons que y est votre vecteur de labels
X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], y, test_size=0.2, random_state=42)


In [6]:
from torch.utils.data import Dataset, DataLoader
from text_dataset import TextDataset


# Créer des instances de Dataset pour PyTorch
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

# Créer des DataLoader pour PyTorch
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,  # votre Dataset personnalisé pour l'entraînement
    batch_size=32,  # Taille du lot
    shuffle=True,   # Mélange des données à chaque époque
    num_workers=4   # Utilisation de 4 sous-processus pour charger les données
)


In [8]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Définir le modèle et les paramètres d'entraînement
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(combined_model.parameters(), lr=0.001)

# Boucle d'entraînement
for epoch in range(8):  # Modifier le nombre d'époques si nécessaire
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # Si votre modèle attend des tokens plutôt que des phrases brutes, vous devez tokeniser ici.
        # Par exemple: inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
        
        outputs = combined_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

Epoch 1, Loss: 1.3361645370721817
Epoch 2, Loss: 0.9520346303780873
Epoch 3, Loss: 0.661434571693341
Epoch 4, Loss: 0.37359609454870224
Epoch 5, Loss: 0.1603835804077486
Epoch 6, Loss: 0.10519596920348703
Epoch 7, Loss: 0.05917411423288286
Epoch 8, Loss: 0.04578222093405202
Finished Training


In [9]:
combined_model.eval()  # Mettre le modèle en mode évaluation
total = 0
correct = 0

with torch.no_grad():  # Pas besoin de calculer les gradients lors de l'évaluation
    for inputs, labels in test_loader:  # test_loader doit être défini comme DataLoader pour vos données de test
        outputs = combined_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on test data: {100 * correct / total}%')


Accuracy of the model on test data: 51.875%


Accuracy of the model on test data: 50.625%


In [10]:
model_path = "path_to_save_model.pth"
torch.save(combined_model.state_dict(), model_path)

In [11]:
combined_model.load_state_dict(torch.load(model_path))
combined_model.eval()  # Ne pas oublier de mettre le modèle en mode évaluation


CombinedModel(
  (sentence_model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 514, 'do_lower_case': False}) with Transformer model: CamembertModel 
    (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (classifier): SimpleClassifier(
    (fc): Linear(in_features=1024, out_features=6, bias=True)
  )
)